In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt    

%matplotlib inline

### 데이터 불러오기

In [4]:
df = pd.read_csv("./loan_data.csv")
df.dataframeName = "loan_data.csv"

### 데이터 살펴보기

In [5]:
df.head(5)

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [12]:
# print(df.shape)
# print(df.dtypes)

In [13]:
# df.describe().round(2)

In [14]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

### 결측값 처리

In [6]:
# 결측값 없기 때문에 처리하지 않음
df.isnull().sum()

credit.policy        0
purpose              0
int.rate             0
installment          0
log.annual.inc       0
dti                  0
fico                 0
days.with.cr.line    0
revol.bal            0
revol.util           0
inq.last.6mths       0
delinq.2yrs          0
pub.rec              0
not.fully.paid       0
dtype: int64

### 범주형 데이터 인코딩

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded_purpose = le.fit_transform(df['purpose'])
df['purpose'] = encoded_purpose
df

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,2,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,1,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,2,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,2,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,1,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,0,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,0,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,2,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,4,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


### 훈련/테스트 데이터 분리

In [8]:
from sklearn.model_selection import train_test_split

X, y = df.drop(labels="not.fully.paid", axis=1), df["not.fully.paid"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

### 데이터 정규화/표준화

In [9]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

print(np.mean(X_train_std), np.std(X_train_std))

6.70169507062147e-17 1.0


In [19]:
# from sklearn.preprocessing import MinMaxScaler

# mc = MinMaxScaler()
# mc.fit(X_train)
# X_train_minmax = sc.transform(X_train)
# X_test_minmax = sc.transform(X_test)

# print(np.mean(X_train_minmax), np.std(X_test_minmax))

### 모델링

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [21]:
reg = MLPClassifier()
reg.fit(X_train_std, y_train)
y_pred = reg.predict(X_test_std)
accuracy_score(y_pred, y_test)

/Users/yeseo/miniconda/envs/ame/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8329853862212944

### 파라미터 튜닝

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import itertools

In [163]:
# 단일 은닉층 (100,)
param_grid = [
    {
        "solver": ["lbfgs", "sgd", "adam"],
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "learning_rate_init": [0.001, 0.01, 0.1],
    },
]

mlp_reg = MLPClassifier(max_iter=5000, early_stopping=True, random_state=42)

# GridSearchCV: 최적의 하이퍼파라미터 조합을 탐색
grid_search = GridSearchCV(
    mlp_reg,
    param_grid,
    # cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="accuracy",
    return_train_score=True,
)

grid_search.fit(X_train_std, y_train)

/Users/yeseo/miniconda/envs/ame/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/yeseo/miniconda/envs/ame/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/yeseo/miniconda/envs/ame/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs

GridSearchCV(estimator=MLPClassifier(early_stopping=True, max_iter=5000,
                                     random_state=42),
             param_grid=[{'activation': ['identity', 'logistic', 'tanh',
                                         'relu'],
                          'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
                          'learning_rate': ['constant', 'invscaling',
                                            'adaptive'],
                          'learning_rate_init': [0.001, 0.01, 0.1],
                          'solver': ['lbfgs', 'sgd', 'adam']}],
             return_train_score=True, scoring='accuracy')

In [164]:
# 최적의 하이퍼파라미터 조합
grid_search.best_params_

{'activation': 'tanh',
 'alpha': 10,
 'learning_rate': 'invscaling',
 'learning_rate_init': 0.01,
 'solver': 'sgd'}

In [165]:
# 최적의 추정기
grid_search.best_estimator_
accuracy_score(grid_search.best_estimator_.predict(X_test_std), y_test)

0.837508698677801

In [166]:
# 평가 점수 확인
cvres = grid_search.cv_results_
cvres
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8399465758456042 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'lbfgs'}
0.8402446381070041 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'sgd'}
0.839498925951896 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'adam'}
0.8399465758456042 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'lbfgs'}
0.8396480683829177 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'sgd'}
0.8387533250971095 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'adam'}
0.8399465758456042 {'activation': 'identity', 'alpha': 0.0001, 'learning_rate': 'constant', 'learning_rate_init': 0.1, 'solver': 'lbfgs'}
0.839051832559796 {'activation':

In [100]:
from sklearn.model_selection import cross_val_score, cross_validate
scores = cross_val_score(
    grid_search.best_estimator_, X_test_std, y_test, scoring="accuracy", cv=5
)
scores

array([0.83652174, 0.83652174, 0.83652174, 0.83826087, 0.83275261])

In [11]:
hidden_layer_sizes_2 = list(tuple(itertools.combinations([2**x for x in range(4, 10)], 2)))
alphas = list(np.logspace(-1, 1, 5))
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [2**x for x in range(4, 10,2)]
len(hidden_layer_sizes_2) * len(alphas) * len(learning_rates) * len(batch_sizes)

675

In [188]:
from sklearn.model_selection import GridSearchCV

# 2개의 은닉층
param_grid = [
    {
        "hidden_layer_sizes": hidden_layer_sizes_2,
        "alpha": alphas,
        "learning_rate_init": learning_rates,
        "batch_size": batch_sizes,
    },
]

mlp_reg = MLPClassifier(
    solver="adam", 
    activation="relu", 
    max_iter=5000, 
    early_stopping=True,
    random_state=42
)

# GridSearchCV: 최적의 하이퍼파라미터 조합을 탐색
grid_search2 = GridSearchCV(
    mlp_reg,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="accuracy",
    return_train_score=True,
)

grid_search2.fit(X_train_std, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPClassifier(early_stopping=True, max_iter=5000,
                                     random_state=42),
             param_grid=[{'alpha': [0.1, 0.31622776601683794, 1.0,
                                    3.1622776601683795, 10.0],
                          'batch_size': [16, 64, 256],
                          'hidden_layer_sizes': [(16, 32), (16, 64), (16, 128),
                                                 (16, 256), (16, 512), (32, 64),
                                                 (32, 128), (32, 256),
                                                 (32, 512), (64, 128),
                                                 (64, 256), (64, 512),
                                                 (128, 256), (128, 512),
                                                 (256, 512)],
                          'learning_rate_init': [0.001, 0.01, 0.1]}],
             return_train_score=True, scoring='accuracy')

In [189]:
# 최적의 하이퍼파라미터 조합
grid_search2.best_params_

{'alpha': 0.31622776601683794,
 'batch_size': 256,
 'hidden_layer_sizes': (64, 256),
 'learning_rate_init': 0.001}

In [190]:
# 최적의 추정기
grid_search2.best_estimator_
accuracy_score(grid_search2.best_estimator_.predict(X_test_std), y_test)

0.83785664578984

In [191]:
# 평가 점수 확인
cvres = grid_search2.cv_results_
cvres
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 32), 'learning_rate_init': 0.001}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 32), 'learning_rate_init': 0.01}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 32), 'learning_rate_init': 0.1}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 64), 'learning_rate_init': 0.001}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 64), 'learning_rate_init': 0.01}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 64), 'learning_rate_init': 0.1}
0.8405430342693689 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 128), 'learning_rate_init': 0.001}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 128), 'learning_rate_init': 0.01}
0.8408413191314124 {'alpha': 0.1, 'batch_size': 16, 'hidden_layer_sizes': (16, 128), 'learning_rate_i

In [12]:
hidden_layer_sizes_3 = list(
    tuple(itertools.combinations([2**x for x in range(4, 10)], 3))
)
len(hidden_layer_sizes_3) * len(alphas) * len(learning_rates) * len(batch_sizes)

900

In [15]:
from sklearn.model_selection import GridSearchCV

# 3개의 은닉층
param_grid = [
    {
        "hidden_layer_sizes": hidden_layer_sizes_3,
        "alpha": alphas,
        "learning_rate_init": learning_rates,
        "batch_size": batch_sizes,
    },
]

mlp_reg = MLPClassifier(max_iter=5000, solver="sgd", activation="relu")

# GridSearchCV: 최적의 하이퍼파라미터 조합을 탐색
grid_search3 = GridSearchCV(
    mlp_reg,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="accuracy",
    return_train_score=True,
)

grid_search3.fit(X_train_std, y_train)

In [ ]:
# 최적의 하이퍼파라미터 조합
grid_search3.best_params_

{'hidden_layer_sizes': (75, 50, 25)}

In [ ]:
# 최적의 추정기
grid_search3.best_estimator_
accuracy_score(grid_search3.best_estimator_.predict(X_test_std), y_test)

0.837508698677801

In [ ]:
# 평가 점수 확인
cvres = grid_search3.cv_results_
cvres
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.840841430431734 {'hidden_layer_sizes': (50, 50, 50)}
0.8411398265940988 {'hidden_layer_sizes': (75, 75, 75)}
0.8400953843756609 {'hidden_layer_sizes': (100, 100, 100)}
0.8411396039934556 {'hidden_layer_sizes': (50, 50, 25)}
0.8412887464244772 {'hidden_layer_sizes': (50, 25, 25)}
0.8399463532449609 {'hidden_layer_sizes': (75, 75, 50)}
0.8402447494073257 {'hidden_layer_sizes': (75, 75, 25)}
0.8403937805380257 {'hidden_layer_sizes': (75, 50, 50)}
0.8414377775551772 {'hidden_layer_sizes': (75, 50, 25)}
0.8397973221142608 {'hidden_layer_sizes': (75, 25, 25)}
0.8230931472391955 {'hidden_layer_sizes': (100, 100, 75)}
0.8402450833082907 {'hidden_layer_sizes': (100, 100, 50)}
0.8113107838881655 {'hidden_layer_sizes': (100, 100, 25)}
0.8116091800505304 {'hidden_layer_sizes': (100, 75, 75)}
0.8087755851614411 {'hidden_layer_sizes': (100, 75, 50)}
0.8238390819949469 {'hidden_layer_sizes': (100, 75, 25)}
0.8397973221142608 {'hidden_layer_sizes': (100, 50, 50)}
0.7956510512315381 {'hidden_layer_si